In [44]:
import os
from langchain_google_genai.llms import GoogleGenerativeAI
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_core.runnables.passthrough import RunnablePassthrough
from langchain.memory.buffer import ConversationBufferMemory
from dotenv import load_dotenv, find_dotenv

In [45]:
load_dotenv(find_dotenv())

True

In [47]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [48]:
EXAMPLES = [
    {
        "query": "Product build for consumers which involes of processing of personal data",
        "answer": "Data security and Privacy rights.",
        "explanation": """
        Since personal data being collected, people have the right to know how we are collecting their personal data and how long we are gonna store it. 
        We also need to make sure proper measures are in place to protect their data and we have proper encrpytion in place
        """
    },
    {
        "query": "Product which requires collection of personal data and sends to third parties.",
        "answer": "Data security, Privacy rights, Lawful basis and transparency and Accountability and governance",
        "explanation": """
        Application collects personal data, so we need to follow privacy rights guideline as people has the right to know that 
        their personal data is collected by the application and we need to make measure to encrpyt the data and protect it, which requires us to follow             Data Security guideline. Lawful basis and transparency is required as we have to make sure who has access to this data and what information 
        we are processing and provide justification for it. To make sure GDPR compilance is following we need to appoint a person in 
        organisation and sign a aggrement between our organisation and third party which require us to follow Accountability and governance guideline.
        """
    }
]

In [49]:
prompt = PromptTemplate(
    template="""
    Role: An Expert in General Data Protection Regulation (GDPR)  guidelines which tells which guideline(s) to follow when building an application or           product.
    Instructions: Answer user query which guide line to follow. Take help from provided examples and don't make up any answer from your side. First             completely analyse the question and properly think befor giving answer and only provide guideline name(s).
    Question: {query}
    Answer: {answer}
    Explanation: {explanation}
    """,
    input_variables=["query", "answer", "explanation"]
)

In [50]:
few_shot_template = FewShotPromptTemplate(
    examples=EXAMPLES,
    example_prompt=prompt,
    suffix="Question: {query}",
    input_variables=["query"]
)

In [51]:
print(few_shot_template.format(query="Ask any question"))


    Role: An Expert in General Data Protection Regulation (GDPR)  guidelines which tells which guideline(s) to follow when building an application or           product.
    Instructions: Answer user query which guide line to follow. Take help from provided examples and don't make up any answer from your side. First             completely analyse the question and properly think befor giving answer and only provide guideline name(s).
    Question: Product build for consumers which involes of processing of personal data
    Answer: Data security and Privacy rights.
    Explanation: 
        Since personal data being collected, people have the right to know how we are collecting their personal data and how long we are gonna store it. 
        We also need to make sure proper measures are in place to protect their data and we have proper encrpytion in place
        
    


    Role: An Expert in General Data Protection Regulation (GDPR)  guidelines which tells which guideline(s) to follow 

In [52]:
google_llm = GoogleGenerativeAI(model="gemini-1.5-pro-latest", temperature=0.4)

In [53]:
chain = (
    {"query": RunnablePassthrough()}
    | few_shot_template
    | google_llm
)

In [54]:
ans = chain.invoke(
    """
    We are building an online book store through which person can order book. For that we collect 
    personal information like person's name, address, phone number. Let me know which GDPR guidelines I need to follow?
    """
)

print(ans)

Answer: Data security, Privacy rights, Lawful basis and transparency 



In [55]:
result = chain.invoke(
    """
    A product which collects personal information like person's name, address, phone number and sends to third party for data processing.    
    Suggest the guideline names which I need to follow?
    """
)

print(result)

Answer: Data security, Privacy rights, Lawful basis and transparency, Accountability and governance, Contracts and data sharing. 



In [56]:
res = chain.invoke(
    """
    A product which involves third party for data processing.Name the GDPR guidelines which I need to follow?
    """
)

print(res)

Answer: Accountability and governance, Data security, Lawful basis and transparency. 

